In [1]:
from datasets import load_dataset
import pandas as pd
import os
from dotenv import load_dotenv
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import pandas
import json
import re
import numpy as np
import openai


from sklearn.metrics.pairwise import cosine_similarity
from pprint import pprint
from pathlib import Path
import random

from langchain.embeddings import OpenAIEmbeddings

### Loading datasets

In [2]:
dataset_lit = load_dataset("jerteh/SrpELTeC")

In [3]:
dataset_lit

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 157430
    })
})

In [4]:
lit_large = dataset_lit['train']['text']

In [5]:
print(lit_large[:5])

['', '<s>10 SRPSKA KNjIŽEVNA ZADRUGA 10</s>', '<s>DVA IDOLA</s>', '<s>BOGOBOJ ATANACKOVIĆ</s>', '<s>U BEOGRADU</s>']


In [6]:
lit_large.index("<s>POP ĆIRA I POP SPIRA</s>")

59300

In [7]:
text_lit = lit_large[59300 : 59300+3500]

In [8]:
len(text_lit)

3500

In [9]:
dataset_wiki = load_dataset("jerteh/SrpWiki")

In [10]:
dataset_wiki

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 3818535
    })
})

In [11]:
text_wiki = dataset_wiki['train']['text']
print(text_wiki[:10])
print(len(text_wiki))
print(type(text_wiki))

['UTF-8 UTF-8 varijanta je najzgodnija za kodiranje većinski latiničnog teksta.', 'Dato je i kratko uputstvo za korišćenje te varijante u Microsoft Word-u, Netscape Composer-u i tekstualnom editoru Kate.', 'U tekstu su takođe preporučeni standardni Unicode fontovi koji omogućavaju laku prenosivost teksta sa računara na računar ili za objavljivanje teksta na Internet.', 'Prvi računari su bili pravljeni pretežno za englesko govorno područje i imali su podršku samo za engleski alfabet, za brojeve, zagrade i još po neki kontrolni karakter, što je činilo ukupno 128 mogućih slova (u 7 bita).', 'To je bio tzv. "-{ASCII}-" ili "-{US-ASCII}-" standard.', '1968. godine je skup karaktera proširen na 256 (8 bita), a "gornjih" 128 karaktera je bilo korišćeno za dodatne karaktere.', 'Iz neke navike je i ovaj prošireni "-{ASCII}-" nazivan "-{ASCII}-", tako da tu često dolazi do zabune.', 'Da bi postojala podrška za više jezika, smišljane su tzv. kodne strane ("-{Code Page}-") koje definišu ponašanje 

In [12]:
dataset_news = load_dataset("jerteh/SrpKorNews")

In [13]:
dataset_news

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 337
    })
})

In [14]:
text_news = dataset_news['train']['text']
print(len(text_news))
print(type(text_news))

337
<class 'list'>


In [15]:
dataset_sci = load_dataset("procesaur/STARS")

In [16]:
dataset_sci

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 7559526
    })
})

In [17]:
text_sci = dataset_sci['train']['text']

In [18]:
print(text_sci[:5])
print(len(text_sci))
print(type)

['<s>Uticaj brzine vazduha na aerosole formirane od 1 % emulzija 112 Uticaj brzine vazduha na aerosole formirane od 6 % emulzija 119 Uticaj brzine vazduha na aerosole formirane od 10 % emulzija 128 Strujanje aerosola vode u cevi</s>', '<s>Završna razmatranja strujanja aerosola u cevi formiranog emulzije EO3</s>', '<s>Moguće strukture aerosol čestice NaCl: a) neporozna čvrsta, b) porozna čvrsta, c) čvrsta sa otvorenim džepovima rastovrenog NaCl, d) čvrsta čaura sa vodenim jezgrom, e) čvrsta NaCl čestica sa jezgrom vodenog rastvora NaCl, f) vodena kapljica NaCl.</s><s>Linije na slikama predstavljaju adsorbovanu vodu.</s>', '<s>Atomiziranje centrifugalnom silom kod rotirajućieg atomizer „rotary atomizer“</s>', '<s>Dezintegrisanje tečnog filma u zavisnosti od relativne brzine a) i c) prednji snimak filma b) i d) bočni snimak filma</s>']
7559526
<class 'type'>


In [69]:
news_mini = text_news[:1]
sci_mini = text_sci[:30]

### Creating subsets

In [19]:
def create_sentnecs(texts):
    sentences = []
    for text in texts:
        matches = re.findall(r'<s>(.*?)</s>', text)
        sentences.extend(matches)
    
    
    return sentences

In [20]:
def make_subset_from_all_sentences(all_sentences: list[str], final_lenght: int, chunked_lenght:int, random_step: int, random_step_start: int = 2000):
    start = 0
    final_sentences = []
    while start+chunked_lenght < len(all_sentences) and len(final_sentences) < final_lenght:
        final_sentences.extend(all_sentences[start:start+chunked_lenght])
        start += chunked_lenght + random.randint(random_step_start, random_step)
    return final_sentences

In [21]:
final_wiki = make_subset_from_all_sentences(text_wiki, 8000, 30, 10_000)

In [22]:
print(final_wiki[:5])
print(len(final_wiki))

['UTF-8 UTF-8 varijanta je najzgodnija za kodiranje većinski latiničnog teksta.', 'Dato je i kratko uputstvo za korišćenje te varijante u Microsoft Word-u, Netscape Composer-u i tekstualnom editoru Kate.', 'U tekstu su takođe preporučeni standardni Unicode fontovi koji omogućavaju laku prenosivost teksta sa računara na računar ili za objavljivanje teksta na Internet.', 'Prvi računari su bili pravljeni pretežno za englesko govorno područje i imali su podršku samo za engleski alfabet, za brojeve, zagrade i još po neki kontrolni karakter, što je činilo ukupno 128 mogućih slova (u 7 bita).', 'To je bio tzv. "-{ASCII}-" ili "-{US-ASCII}-" standard.']
8010


In [23]:
sent_news = create_sentnecs(text_news)
print(sent_news[:5])
print(len(sent_news))

['Mečka na usijanom limenom krovu', 'Piše: Dušan Vidaković Kada je austrijski konzul u Nici Emil Jelinek 1901. naručio od nemačkog Dajmlera, odjednom, čak 36 automobila, čija je ukupna cena premašila fantastičnih 550 hiljada zlatnih maraka, imao je samo jedan uslov: da sva ta kola nose ime njegove ćerke.', 'Ljudi iz kanstatske firme, naravno, nisu imali ništa protiv, a istorija automobilizma je imala sreću da je Jelinekovoj mezimici kum dao (pogotovo za Špance) vrlo graciozno ime - Mercedes, što na jeziku konkvistadora znaci ljupkost, umilnost, prijatnost.', 'Jelinekova porudžbina značila prekretnicu u poslovanju Dajmlera, a naziv Mercedes je preživeo i fuziju sa fabrikom Karla Benca, 1926. godine, održavši se do danas kao sinonim za kvalitetne, luksuzne i, nadasve, bezbedne automobile svojstvenih vozačima koji ne kriju brige u novčaniku.', 'Međutim, upravo legenda o blistavosti trokrake zvezde doživljava ovih dana ozbiljnu proveru zahvaljujući mercedesu A klase.']
21982152


In [24]:
final_news = make_subset_from_all_sentences(sent_news, 2000, 30, 100_000)

In [54]:
print(final_news[:5])
print(len(final_news))

['Mečka na usijanom limenom krovu', 'Piše: Dušan Vidaković Kada je austrijski konzul u Nici Emil Jelinek 1901. naručio od nemačkog Dajmlera, odjednom, čak 36 automobila, čija je ukupna cena premašila fantastičnih 550 hiljada zlatnih maraka, imao je samo jedan uslov: da sva ta kola nose ime njegove ćerke.', 'Ljudi iz kanstatske firme, naravno, nisu imali ništa protiv, a istorija automobilizma je imala sreću da je Jelinekovoj mezimici kum dao (pogotovo za Špance) vrlo graciozno ime - Mercedes, što na jeziku konkvistadora znaci ljupkost, umilnost, prijatnost.', 'Jelinekova porudžbina značila prekretnicu u poslovanju Dajmlera, a naziv Mercedes je preživeo i fuziju sa fabrikom Karla Benca, 1926. godine, održavši se do danas kao sinonim za kvalitetne, luksuzne i, nadasve, bezbedne automobile svojstvenih vozačima koji ne kriju brige u novčaniku.', 'Međutim, upravo legenda o blistavosti trokrake zvezde doživljava ovih dana ozbiljnu proveru zahvaljujući mercedesu A klase.']
2010


In [26]:
removed_lit = create_sentnecs(text_lit)
print(removed_lit[:5])
print(len(removed_lit))

['POP ĆIRA I POP SPIRA', 'POP ĆIRA I POP SPIRA', 'STEVANA SREMAC', 'IZDANjE BRANKOVA KOLA', 'SVOME UJAKU']
8639


In [27]:
final_lit = make_subset_from_all_sentences(removed_lit, 1250, 30, 100, random_step_start=0)

In [28]:
print(final_lit[:30])
print(len(final_lit))

['POP ĆIRA I POP SPIRA', 'POP ĆIRA I POP SPIRA', 'STEVANA SREMAC', 'IZDANjE BRANKOVA KOLA', 'SVOME UJAKU', 'JOVANU ĐORĐEVIĆU', 'PROFESORU VELIKE ŠKOLE', 'ODŠTAMPANO IZ BRANKOVA KOLA', 'U kojoj su opisana dva popa, dve popadije i dve popine ćerke iz jednoga sela u Banatu, u kome su parohijani bili tako pobožni, da su badava mleli svojim popovima brašno u suvačama', 'Bila dva popa, — ali ne ona dva popa, što su jednom ostali sami u svetu, pa se svaki od njih tužio i tešio, da bi mu daleko lakše bilo životariti, samo da nema onoga drugog, — ne, dakle, ta dva popa, nego druga dva, i živela su u jednom selu u Banatu.', 'Koje je to selo, nećemo vam kazati, da ne bi selo, ni krivo ni dužno, potrzali i izlagali ga podsmehu, pošto ono nije ni najmanje krivo za sve ovo što će se u ovoj pripovetci pričati.', 'A posle, ako ćemo šta, ono i nije selo, nego varošica.', 'Ta kolik je samo onaj Veliki Sokak s kraja na kraj, pa koliko tek malih sokaka ima, a svi su široki.', 'Zato ni jedan nije kaldrmisa

In [43]:
removed_sci = create_sentnecs(text_sci[:450_000])
print(removed_sci[:5])
print(len(removed_sci))

['Uticaj brzine vazduha na aerosole formirane od 1 % emulzija 112 Uticaj brzine vazduha na aerosole formirane od 6 % emulzija 119 Uticaj brzine vazduha na aerosole formirane od 10 % emulzija 128 Strujanje aerosola vode u cevi', 'Završna razmatranja strujanja aerosola u cevi formiranog emulzije EO3', 'Moguće strukture aerosol čestice NaCl: a) neporozna čvrsta, b) porozna čvrsta, c) čvrsta sa otvorenim džepovima rastovrenog NaCl, d) čvrsta čaura sa vodenim jezgrom, e) čvrsta NaCl čestica sa jezgrom vodenog rastvora NaCl, f) vodena kapljica NaCl.', 'Linije na slikama predstavljaju adsorbovanu vodu.', 'Atomiziranje centrifugalnom silom kod rotirajućieg atomizer „rotary atomizer“']
1467642


In [50]:
final_sci=make_subset_from_all_sentences(removed_sci, 1250, 30, 10_000)
print(final_sci)
print(len(final_sci))

['Uticaj brzine vazduha na aerosole formirane od 1 % emulzija 112 Uticaj brzine vazduha na aerosole formirane od 6 % emulzija 119 Uticaj brzine vazduha na aerosole formirane od 10 % emulzija 128 Strujanje aerosola vode u cevi', 'Završna razmatranja strujanja aerosola u cevi formiranog emulzije EO3', 'Moguće strukture aerosol čestice NaCl: a) neporozna čvrsta, b) porozna čvrsta, c) čvrsta sa otvorenim džepovima rastovrenog NaCl, d) čvrsta čaura sa vodenim jezgrom, e) čvrsta NaCl čestica sa jezgrom vodenog rastvora NaCl, f) vodena kapljica NaCl.', 'Linije na slikama predstavljaju adsorbovanu vodu.', 'Atomiziranje centrifugalnom silom kod rotirajućieg atomizer „rotary atomizer“', 'Dezintegrisanje tečnog filma u zavisnosti od relativne brzine a) i c) prednji snimak filma b) i d) bočni snimak filma', 'Režimi dezintegrisanja filmova: a) kolutasto/prstenast b) talasast i c) perforiran', 'Dva tipa talasa koja se mogu formirati pri dezintegrisanju tečnih filmova', 'Grafik zavisnosti Reynolds-ov

### Creating sentences

In [45]:
# crate one function with create_sentences and flag for loop if dataset isWiki? 
def return_dic(sentences):
    sent_lst_dic = [{'sentence': x, 'id' : i} for i, x in enumerate(sentences)]
    return sent_lst_dic

In [46]:
sent_wiki = return_dic(final_wiki)
pprint(sent_wiki[:3])
print(len(sent_wiki))

[{'id': 0,
  'sentence': 'UTF-8 UTF-8 varijanta je najzgodnija za kodiranje većinski '
              'latiničnog teksta.'},
 {'id': 1,
  'sentence': 'Dato je i kratko uputstvo za korišćenje te varijante u '
              'Microsoft Word-u, Netscape Composer-u i tekstualnom editoru '
              'Kate.'},
 {'id': 2,
  'sentence': 'U tekstu su takođe preporučeni standardni Unicode fontovi koji '
              'omogućavaju laku prenosivost teksta sa računara na računar ili '
              'za objavljivanje teksta na Internet.'}]
8010


In [47]:
sent_news = return_dic(final_news)
pprint(sent_news[:3])
print(len(sent_news))

[{'id': 0, 'sentence': 'Mečka na usijanom limenom krovu'},
 {'id': 1,
  'sentence': 'Piše: Dušan Vidaković Kada je austrijski konzul u Nici Emil '
              'Jelinek 1901. naručio od nemačkog Dajmlera, odjednom, čak 36 '
              'automobila, čija je ukupna cena premašila fantastičnih 550 '
              'hiljada zlatnih maraka, imao je samo jedan uslov: da sva ta '
              'kola nose ime njegove ćerke.'},
 {'id': 2,
  'sentence': 'Ljudi iz kanstatske firme, naravno, nisu imali ništa protiv, a '
              'istorija automobilizma je imala sreću da je Jelinekovoj '
              'mezimici kum dao (pogotovo za Špance) vrlo graciozno ime - '
              'Mercedes, što na jeziku konkvistadora znaci ljupkost, umilnost, '
              'prijatnost.'}]
2010


In [51]:
sent_sci = return_dic(final_sci)
pprint(sent_sci[:3])
print(len(sent_sci))

[{'id': 0,
  'sentence': 'Uticaj brzine vazduha na aerosole formirane od 1 % emulzija 112 '
              'Uticaj brzine vazduha na aerosole formirane od 6 % emulzija 119 '
              'Uticaj brzine vazduha na aerosole formirane od 10 % emulzija '
              '128 Strujanje aerosola vode u cevi'},
 {'id': 1,
  'sentence': 'Završna razmatranja strujanja aerosola u cevi formiranog '
              'emulzije EO3'},
 {'id': 2,
  'sentence': 'Moguće strukture aerosol čestice NaCl: a) neporozna čvrsta, b) '
              'porozna čvrsta, c) čvrsta sa otvorenim džepovima rastovrenog '
              'NaCl, d) čvrsta čaura sa vodenim jezgrom, e) čvrsta NaCl '
              'čestica sa jezgrom vodenog rastvora NaCl, f) vodena kapljica '
              'NaCl.'}]
1260


In [53]:
sent_lit = return_dic(final_lit)
pprint(sent_lit[:10])
print(len(sent_lit))

[{'id': 0, 'sentence': 'POP ĆIRA I POP SPIRA'},
 {'id': 1, 'sentence': 'POP ĆIRA I POP SPIRA'},
 {'id': 2, 'sentence': 'STEVANA SREMAC'},
 {'id': 3, 'sentence': 'IZDANjE BRANKOVA KOLA'},
 {'id': 4, 'sentence': 'SVOME UJAKU'},
 {'id': 5, 'sentence': 'JOVANU ĐORĐEVIĆU'},
 {'id': 6, 'sentence': 'PROFESORU VELIKE ŠKOLE'},
 {'id': 7, 'sentence': 'ODŠTAMPANO IZ BRANKOVA KOLA'},
 {'id': 8,
  'sentence': 'U kojoj su opisana dva popa, dve popadije i dve popine ćerke iz '
              'jednoga sela u Banatu, u kome su parohijani bili tako pobožni, '
              'da su badava mleli svojim popovima brašno u suvačama'},
 {'id': 9,
  'sentence': 'Bila dva popa, — ali ne ona dva popa, što su jednom ostali sami '
              'u svetu, pa se svaki od njih tužio i tešio, da bi mu daleko '
              'lakše bilo životariti, samo da nema onoga drugog, — ne, dakle, '
              'ta dva popa, nego druga dva, i živela su u jednom selu u '
              'Banatu.'}]
1260


In [ ]:
print(sum(len(sent)))

### Creating combined sentences

Source: https://github.com/FullStackRetrieval-com/RetrievalTutorials/blob/8a30b5710b3dd99ef2239fb60c7b54bc38d3613d/tutorials/LevelsOfTextSplitting/5_Levels_Of_Text_Splitting.ipynb

In [101]:
sent_wiki_mini = sent_wiki[:10]
sent_wiki_mini

[{'sentence': 'UTF-8 UTF-8 varijanta je najzgodnija za kodiranje većinski latiničnog teksta.',
  'id': 0},
 {'sentence': 'Dato je i kratko uputstvo za korišćenje te varijante u Microsoft Word-u, Netscape Composer-u i tekstualnom editoru Kate.',
  'id': 1},
 {'sentence': 'U tekstu su takođe preporučeni standardni Unicode fontovi koji omogućavaju laku prenosivost teksta sa računara na računar ili za objavljivanje teksta na Internet.',
  'id': 2},
 {'sentence': 'Prvi računari su bili pravljeni pretežno za englesko govorno područje i imali su podršku samo za engleski alfabet, za brojeve, zagrade i još po neki kontrolni karakter, što je činilo ukupno 128 mogućih slova (u 7 bita).',
  'id': 3},
 {'sentence': 'To je bio tzv. "-{ASCII}-" ili "-{US-ASCII}-" standard.',
  'id': 4},
 {'sentence': '1968. godine je skup karaktera proširen na 256 (8 bita), a "gornjih" 128 karaktera je bilo korišćeno za dodatne karaktere.',
  'id': 5},
 {'sentence': 'Iz neke navike je i ovaj prošireni "-{ASCII}-" naz

In [55]:
openai.api_key = os.getenv("OPENAI_API_KEY")
oaiembeds = OpenAIEmbeddings()

/home/teodora/Development/SRBedding/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [56]:
buffer_size = 1

In [57]:
# can this be improved?
def combine_sentences(sentences, buffer_size=buffer_size):
    # Go through each sentence dict
    for i in range(len(sentences)):

        # Create a string that will hold the sentences which are joined
        combined_sentence = ''

        # Add sentences before the current one, based on the buffer size.
        for j in range(i - buffer_size, i):
            # Check if the index j is not negative (to avoid index out of range like on the first one)
            if j >= 0:
                # Add the sentence at index j to the combined_sentence string
                combined_sentence += sentences[j]['sentence'] + ' '

        # Add the current sentence
        combined_sentence += sentences[i]['sentence']

        # Add sentences after the current one, based on the buffer size
        for j in range(i + 1, i + 1 + buffer_size):
            # Check if the index j is within the range of the sentences list
            if j < len(sentences):
                # Add the sentence at index j to the combined_sentence string
                combined_sentence += ' ' + sentences[j]['sentence']

        # Then add the whole thing to your dict
        # Store the combined sentence in the current sentence dict
        sentences[i]['combined_sentence'] = combined_sentence

    return sentences

In [103]:
cobined_mini = combine_sentences(sent_wiki_mini)
cobined_mini

[{'sentence': 'UTF-8 UTF-8 varijanta je najzgodnija za kodiranje većinski latiničnog teksta.',
  'id': 0,
  'combined_sentence': 'UTF-8 UTF-8 varijanta je najzgodnija za kodiranje većinski latiničnog teksta. Dato je i kratko uputstvo za korišćenje te varijante u Microsoft Word-u, Netscape Composer-u i tekstualnom editoru Kate.'},
 {'sentence': 'Dato je i kratko uputstvo za korišćenje te varijante u Microsoft Word-u, Netscape Composer-u i tekstualnom editoru Kate.',
  'id': 1,
  'combined_sentence': 'UTF-8 UTF-8 varijanta je najzgodnija za kodiranje većinski latiničnog teksta. Dato je i kratko uputstvo za korišćenje te varijante u Microsoft Word-u, Netscape Composer-u i tekstualnom editoru Kate. U tekstu su takođe preporučeni standardni Unicode fontovi koji omogućavaju laku prenosivost teksta sa računara na računar ili za objavljivanje teksta na Internet.'},
 {'sentence': 'U tekstu su takođe preporučeni standardni Unicode fontovi koji omogućavaju laku prenosivost teksta sa računara na r

In [58]:
def generate_embeddings(sentences):
    embeddings = oaiembeds.embed_documents([x['combined_sentence'] for x in sentences])
    for i, sentence in enumerate(sentences):
        sentence['combined_sentence_embedding'] = embeddings[i]
    
    return sentences

In [104]:
embeddings_mini = generate_embeddings(cobined_mini)
embeddings_mini

[{'sentence': 'UTF-8 UTF-8 varijanta je najzgodnija za kodiranje većinski latiničnog teksta.',
  'id': 0,
  'combined_sentence': 'UTF-8 UTF-8 varijanta je najzgodnija za kodiranje većinski latiničnog teksta. Dato je i kratko uputstvo za korišćenje te varijante u Microsoft Word-u, Netscape Composer-u i tekstualnom editoru Kate.',
  'combined_sentence_embedding': [0.003918936192710906,
   -0.004485567707291817,
   -0.007685926652700066,
   -0.03684686230393266,
   0.017676365214827648,
   -0.0016175899649009003,
   -0.03036384251182653,
   -0.006698278691516845,
   -0.0178916217859321,
   -0.023792185031413844,
   -0.0031212205138451783,
   0.02998397655020778,
   -0.008312703440461488,
   0.004821114359096034,
   -0.009237040097600749,
   0.011554214267837057,
   0.02747687126180711,
   -0.00404555786947632,
   0.006995839492217192,
   -0.035681945485393864,
   -0.00847731096833077,
   0.03601116054113243,
   -0.011801126490963489,
   -0.008420331726013712,
   -0.01254186222902028,
   0

In [59]:
# why convert cosine similarity to cosine distance?
# just use cosine similarity instead?
# test on couple of examples and see if it makes a difference
def calculate_cosine_distances(sentences):
    distances = []
    for i in range(len(sentences) - 1):
        embedding_current = sentences[i]['combined_sentence_embedding']
        embedding_next = sentences[i + 1]['combined_sentence_embedding']
        
        # Calculate cosine similarity
        similarity = cosine_similarity([embedding_current], [embedding_next])[0][0]
        
        # Convert to cosine distance
        distance = 1 - similarity

        # Append cosine distance to the list
        distances.append(distance)

        # Store distance in the dictionary
        sentences[i]['distance_to_next'] = distance

    # Optionally handle the last sentence
    # sentences[-1]['distance_to_next'] = None  # or a default value

    return distances, sentences

In [108]:
distances_mini, sentences_mini = calculate_cosine_distances(embeddings_mini)
distances_mini

[0.0068540953077664835,
 0.04575485248702682,
 0.030882957162866154,
 0.041640690866002306,
 0.04913022845713788,
 0.02850502194662452,
 0.05728326745292067,
 0.028669636838056345,
 0.03379749834565815]

In [109]:
def get_breakpoint(sentences, distances, treshold):
    # We need to get the distance threshold that we'll consider an outlier
    # We'll use numpy .percentile() for this
    breakpoint_distance_threshold = np.percentile(distances, treshold) # If you want more chunks, lower the percentile cutoff

    # Then we'll see how many distances are actually above this one
    #num_distances_above_theshold = len([x for x in distances if x > breakpoint_distance_threshold]) # The amount of distances above your threshold

    # Then we'll get the index of the distances that are above the threshold. This will tell us where we should split our text
    indices_above_thresh = [i for i, x in enumerate(distances) if x > breakpoint_distance_threshold] # The indices of those breakpoints on your list

    # Initialize the start index
    start_index = 0

    # Create a list to hold the grouped sentences
    chunks = []

    # Iterate through the breakpoints to slice the sentences
    for index in indices_above_thresh:
        # The end index is the current breakpoint
        end_index = index

        # Slice the sentence_dicts from the current start index to the end index
        group = sentences[start_index:end_index + 1]
        combined_text = ' '.join([d['sentence'] for d in group])
        chunks.append(combined_text)
        
        # Update the start index for the next group
        start_index = index + 1

    # The last group, if any sentences remain
    if start_index < len(sentences):
        combined_text = ' '.join([d['sentence'] for d in sentences[start_index:]])
        chunks.append(combined_text)

    return chunks

In [63]:
combined_lit = combine_sentences(sent_lit)
pprint(combined_lit[20])

{'combined_sentence': 'Osobito u selu ni za živu glavu niko to ne sme da '
                      'spomene; šta više, sad još hvale mesto da je suvo i '
                      'ocedno i da mu i ne treba trotoar. No među tim nije baš '
                      'ni tako. Ispred svake kuće ima jendek pun vode, koji se '
                      'nikad ne prazni; jer taman da presuši a ono po '
                      'Bečkerečkom Velikom Kalendaru udari blagoslov iz neba, '
                      'i jendek se napuni opet kao što je i bio i nudi i mami '
                      'na uživanja i danju i noću.',
 'id': 20,
 'sentence': 'No među tim nije baš ni tako.'}


In [64]:
embeddings_lit = generate_embeddings(combined_lit)
print(embeddings_lit[0])
print(len(embeddings_lit))

{'sentence': 'POP ĆIRA I POP SPIRA', 'id': 0, 'combined_sentence': 'POP ĆIRA I POP SPIRA POP ĆIRA I POP SPIRA', 'combined_sentence_embedding': [-0.010754292333241107, -0.02212568828909579, 0.008645860078251986, -0.025828300711551917, -0.0037026135866092924, 0.0015098114265055529, -0.030186586547942183, -0.01797310366024338, -0.0021662860106474342, -0.01616036630901335, 0.0009521694879009701, 0.002291635038471671, -0.015273280666413553, -0.006148524297903724, 0.015414700130180514, -0.003529053483787516, 0.02465837707274073, -0.011281400396988389, -0.010477882325142301, -0.013151991503876734, -0.003763680870662632, 0.02153429848157761, 0.004223293429413356, -0.012792015787669279, -0.001588556259945079, -0.00047729002082542353, -0.007122388711345929, -0.009783641913855196, 0.00599424839225033, -0.013023428947657472, 0.0023028842504993247, -0.012830584647667311, -0.009275818745768197, -0.010497166289480052, -0.023681298665242233, -0.04689977101050808, -0.012554173708245972, 0.0007565127102

In [67]:
distances_lit, sentences_lit  = calculate_cosine_distances(embeddings_lit)
print(distances_lit)
print(len(distances_lit))

[0.04192271170643502, 0.0452954415968545, 0.07633677073546652, 0.05894970525929499, 0.08595923397347638, 0.048287874563319844, 0.08847912083666509, 0.04459721864177202, 0.024503618743136446, 0.03413939612468697, 0.09631736728252993, 0.05027664406375165, 0.0573618306109589, 0.08079499624720232, 0.013143924418286157, 0.04173893540687246, 0.14128818998273496, 0.07608137152845484, 0.05120176269661747, 0.09251753374524596, 0.05506948061286987, 0.023000577110368292, 0.08229234934649132, 0.044512253555150094, 0.06120733732830275, 0.08107941189204959, 0.04816860550600499, 0.03886802723481453, 0.03984548669076737, 0.05159853770335632, 0.08698822835191056, 0.06838433621721618, 0.08080116920320224, 0.062423080156549626, 0.05119846218656088, 0.05207358827482522, 0.01828221188602064, 0.06313629086300587, 0.07440382953304314, 0.035103947957935366, 0.03674112580738231, 0.06183934038933159, 0.03393462769294664, 0.05588676246994173, 0.05472302078384228, 0.05591844326912021, 0.04247956112823492, 0.04330

In [120]:
breakpoint_lit = get_breakpoint(sentences_lit, distances_lit, 95)
print(len(breakpoint_lit))
breakpoint_lit

64


['POP ĆIRA I POP SPIRA POP ĆIRA I POP SPIRA STEVANA SREMAC IZDANjE BRANKOVA KOLA SVOME UJAKU JOVANU ĐORĐEVIĆU PROFESORU VELIKE ŠKOLE ODŠTAMPANO IZ BRANKOVA KOLA U kojoj su opisana dva popa, dve popadije i dve popine ćerke iz jednoga sela u Banatu, u kome su parohijani bili tako pobožni, da su badava mleli svojim popovima brašno u suvačama Bila dva popa, — ali ne ona dva popa, što su jednom ostali sami u svetu, pa se svaki od njih tužio i tešio, da bi mu daleko lakše bilo životariti, samo da nema onoga drugog, — ne, dakle, ta dva popa, nego druga dva, i živela su u jednom selu u Banatu. Koje je to selo, nećemo vam kazati, da ne bi selo, ni krivo ni dužno, potrzali i izlagali ga podsmehu, pošto ono nije ni najmanje krivo za sve ovo što će se u ovoj pripovetci pričati. A posle, ako ćemo šta, ono i nije selo, nego varošica. Ta kolik je samo onaj Veliki Sokak s kraja na kraj, pa koliko tek malih sokaka ima, a svi su široki. Zato ni jedan nije kaldrmisan, a nije nikada ni bio. Istina, podsme

In [122]:
for i, chunk in enumerate(breakpoint_lit[30:40]):
    buffer = 400
    
    print (f"Chunk #{i}")
    print (chunk)
    print ("\n")

Chunk #0
— A izvetrilo je to čim sam iziš’o na jaroš! A pa onda... ne moramo svaki dan piti, nego da se malo vidimo i razgovaramo — — Jao mene! A kakav te razgovor nap’o sad, naopako ti zvonilo — — Eto ti sad nje, kako samo govori! — Ta da! A kakva bi’ ja mati (jao žalosnoj meni!) bila, da danas idem tamo?! — E, a kao za što da ne idemo? — Ta zar ispred nosa ti ga odvukli pa sad nam se smeju i on i ona beštija čifucka! I mi, mi da idemo!? — Pa šta su ga odvukli? — I odvukli su ga, i dočepali su ga, i neće ga šale pustiti. Ne iskobelja se više taj k’o ni paor iz fiškalskih šaka! — Pa valjda se sad nećemo tući zbog njega?!... — Pa valjda da ostanemo flegmatični. E, Spiro, Spiro, ne nosila ja tu mantiju! — Pa devojačka su vrata svakom otvorena... A on neka bira; ima oči, fala Bogu, nije ćorav, pa neka bira. Valjda mu neću nametati k’o fališnu forintu. — Hu-u-u! Ta što će je izabrati i nije mi za to toliko, al’ što će mi pucati pod nos ona beštija!.. — Ama, Sido, sve mi se čini da ti —


C

— A izvetrilo je to čim sam iziš’o na jaroš! A pa onda... ne moramo svaki dan piti, nego da se malo vidimo i razgovaramo — — Jao mene! A kakav te razgovor nap’o sad, naopako ti zvonilo — — Eto ti sad nje, kako samo govori! — Ta da! A kakva bi’ ja mati (jao žalosnoj meni!) bila, da danas idem tamo?! — E, a kao za što da ne idemo? — Ta zar ispred nosa ti ga odvukli pa sad nam se smeju i on i ona beštija čifucka! I mi, mi da idemo!? — Pa šta su ga odvukli? — I odvukli su ga, i dočepali su ga, i neće ga šale pustiti. Ne iskobelja se više taj k’o ni paor iz fiškalskih šaka! — Pa valjda se sad nećemo tući zbog njega?!... — Pa valjda da ostanemo flegmatični. E, Spiro, Spiro, ne nosila ja tu mantiju! — Pa devojačka su vrata svakom otvorena... A on neka bira; ima oči, fala Bogu, nije ćorav, pa neka bira. Valjda mu neću nametati k’o fališnu forintu. — Hu-u-u! Ta što će je izabrati i nije mi za to toliko, al’ što će mi pucati pod nos ona beštija!.. — Ama, Sido, sve mi se čini da ti —

— Eto! Što ga nema kod nas; a kod njih je bio i pre i posle podne! Doš’o je posle podne još kad je Žuža išla po vešplau, a izaš’o je kad su se krave vraćale kućama! Izašli pred kuću, pa se čitav sat špacirali. A ja je gledam, pa mi se sve privrće u meni i kuva k’o u šporertu. (Spiro, treb’o si viditi, a ja ti ne umem sve kazati!) Špacira se s njim ispred kuće, a iznela veliku loptu što je na prezent dobila od onog Švabe kapetana sas lađe „Maria-Anne“, pa se lopta, pa svaki čas ispusti, k’o bajagi, loptu, a  lopta se otkotrlja, a ona ga onda moli da joj donese, pa kad joj on donese loptu a ona mu zafaluje a sve prevrće s očima i stisne ga za ruku. Kad brž’ to, kaži ti to meni i protolmači?!

A furt se nešto smeju a osobito ona. On i kojekako; snebiva se siroma’ mladić, pa se čudi, šta ga je snašlo. Al’ ona, smeje se, Bože, smeje, smeje, pa misliš sve zvoni sokak od njena smeja! Ona mu nešto govori a sve gleda ovamo na našu kapiju — baš lepo sam vidila kroz izbijen čvor na kapiji — a on se samo smeši. A ona smeje se, Bože, smeje, pa zabacila glavu pa sve baca noge i šora nogama po zemlji, pa digla nos u nebo, misliš Venus je, niko k’o ona! Ta u mal’ joj nisam pritrčala preko sokaka; a kupila bi joj mama kofpuc i hornodle po sokaku a lokne po komšiluku... priselo bi joj i loptanje i smejanje!... Mora da se nešto preko nas podsmevala, ne sme faliti, to je, Spiro, k’o ajmol ajnc; jer što ja vidim to je viđeno, Spiro, pa punktum! — Hm, hm! — vrteo je glavom pop Spira. Prošlo je nekoliko dana posle ovoga, a odnošaji se ne samo da nisu ni malo poboljšali, nego su se sve više i više zapletali. Pera je redovno svaki dan dolazio pop-Ćirinima a tek jedared ili najviše dvared ako je bio kod pop-Spirinih. Gđa Sida je uvidela naskoro, da za njenu Julu nema nikakva izgleda. Jer Jula je i nadalje ostala onako mirna i povučena a posle one scene u pop-Ćirinoj  kući još više. Gđa Sida je bila ljuta i čudno joj je bilo kao što je, znam, čudno i čitaocima, ali pisac obećava, da će skorim svaki čitalac videti u čemu je stvar. Kazaće im on to već na svome mestu, i znam da će im onda još čudnije biti, kako to da se nisu ranije setili, kad je davno u romanima rečeno da se srcu ne da zapovedati a u pesmarici zapisano „Nije blago ni srebro ni zlato, već je blago što je srcu drago.“ Među tim dok je Jula silazila, i kao zvezda zalazila za orizonat, Melanija se sve više penjala. Znala je oko Pere tako vešto i umiljato da se ponaša, da mu je već teško bilo i pola dana bez nje. I ne znam kako, već tada mu je padala na pamet strašna misao: kakva bi to velika nesreća bila, da nešto kao pop ostane udovac da mu — ne daj Bože, — umre taka popadija! Osećao je, da to ne bi bio u stanju preživeti. I slika Melanijina postajala mu je sve dražija. A i ona od svoje strane koliko ga je samo svojim žrtvama obvezala! Uklonila je iz albuma sve muškarce ispod pedeset godina; i hulanerski, i infanteriski, i artelirski, pa čak i „marinerski“ oficir, ustupiše mesta svršenom kliriku Petru Petroviću čije se četir fotografije od sad nalazile u albumu prema njoj. Svaki dan bi im odlazio i donosio knjige koje su zajedno u bašti pod orahom čitali i zajedno uzdisali ako bi što žalostivo bilo; na onim tronljivim mestima Peri bi drhtao glas, a Melanija briše tankom šnuftihlom nos, da  bi predupredila potok suza. Gđa Persa sedi i podštrikava popine čarape, i ona sluša, samo što ona ne plače, ali se ipak čudi, kako to sve tako lepo umeju ljudi da izmisle i sastave, k’o da su baš tamo bili! Posle čitanja čeka ih „jauzn“, a posle ovoga izlaze i šetaju se ispred kuće i razgovaraju, a razgovoru zaljubljenih nikad kraja. I opet izlazi Erža i nudi i gospodina Peru na večeru, a on il’ primi ponudu il’ se učtivo izvini, klanja, rukuje i odlazi, a ona na poluotvorenih vratih stoji i gleda za njim, sve dok ne savije u drugi sokak; a kad savije u drugi sokak, a on još jedared pogleda na vrata, a ona još stoji i gleda za njim, maše mu maramom i ulazi, a on njoj mane šeširom i zalazi u drugi sokak pa sve cupka od miline i nikoga ne vidi, ili se svakom javlja. A gđa Sida se samo krsti  iza one rupe od čvora na kapiji, krsti se i čudi i veli: „O, časni te, devojko!“ Gđa Sida nije mogla to da trpi, smetalo joj je, upravo kvarilo njen dojakošnji red i naviku. I ona je naučila svako predveče da sedne na klupu pred kućom, pa da se javlja mimoprolazećima i da daje savete Juli koja sedi pored nje ili Žuži koja u to doba obično poliva ulicu ili plevi zubaču, da ne bi sedela besposlena, jer besposlenost je, govorila je gđa Sida, izvor tolikim porocima. To je stara njena navika od toliko godina, i sada, gle, mora da je se odvikne od kako je Melanija počela da se šeta ispred kuće! Međutim, da popusti kad je ona u pravu, i kad je ona gazdarica ispred svoje kuće pa sve do pola sokaka — to, Boga mi, nikako nije htela, a nije ni tražiti. Mislila je i mislila, kako bi im doskočila — i na posletku se dosetila, slavno dosetila. U selu nema baš Bog zna koliko policijskih uredaba, upravo njih ima dosta, ali malo ih ima koji ih zarezuju u što. Sokak ili avlija, to je sve jedno, u svakom selu. Za to i u ovom našem nije bila retkost, da se razapne pajvan i da se suši veš preko sokaka. Psuje, istina, svet i pajvan i gazdu, ali opet ostaje sve po starom; kako je kome udesnije tako i radi. Pa kad je to sve tako, onda će vam razumljivo biti i ovo što je gđa Sida sad uradila. A baš joj je zgodno došlo da uradi što je smislila; nije imala koga da pita, nego je mogla sama da uradi,  jer pop Spira ni pop Ćira nije bio u selu. Otišli su kao prekjuče, a baviće se još ne znam koliko koji na putu. Dakle, sve k’o poručeno, živa zgoda. Sutra dan već je bilo namešteno jedno rešeto za žito (vetrenjača) i radili su od rane zore do mrkla mraka. Grdna prašina i pleva letela je i padala, blagodareći pospešnom vetru, na pop-Ćirinu kuću baš kad su izašli da se šetaju, a rešeto je užasno zablebetalo i lupalo, da je Melaniju formalno ućutkalo i ova ne mogaše do reči doći. — A, a, to se ne može izdržati! — viče Melanija. — Zaista ne može. Gotovo da ostavimo šetnju za sutra. Klanjam se, gospođice! — viče Pera. — Službenica!




### Generating queries

In [ ]:
# Selena
SYSTEN_PROMPT_QUERIES = """
### Goal ###
Tre primary objective is to produce multiple queries in the Serbian language and a list of keywords in the Serbian language from the provided context. The context repesents an answer to the query and the keywords best 
describe the context. The goal is to have query-context pairs that corelate with each other and a list of keywords that would spead-up the search in the future.

### Process Overview ###
1. Carefully read and analyze the given context text.
2. Identify all relevant keywords and what the context text is about.
3. Find the queries that best represents the given context text.

### Formatting Rules ###
- Keyword value MUST be a LIST of strings with max lenght of 5 or [null] if no relevant information is provided.
- Use double quotes for strings and escape internal quotes with a backslash (\).
- Keep the queries concise and general about the context text.
- Ensure the output is a valid JSON file, parsable by Python's json.loads().
- Strictly use only the information provided in the context text. Do not add, infer, or imagine any additional details beyond what is explicitly stated.
- Remember to answer in Serbian.

### Query description###
- All queries must be WH-questions.
- All queries must begin with a capital letter and end with a questoin mark (?).

### Output Format ###
{{
 "keywords": ["The keyword that best represent the given context with max lenght of 5"],
 "short_query": "A short query that best suits the given context. It should be only a few words long and general.",
 "medium_query": "A minium lenght query that best suits the given context. It should be a lenght of min 10 words and max 18.",
 "long_query": "A long query that best suits the given context. It should be longer than 19 words and very specific to the context."
}}

### Context ###
{context}
"""

In [ ]:
# Selena
# def test_api(prompt, context, model: str = "gpt-3.5-turbo-0125"):
#     client = openai


#     # Assuming OpenAI API key is set elsewhere
#     # Initiate the streaming response
#     response = client.chat.completions.create(
#         model=model,
#         messages=[
#             {"role": "system", "content": prompt},
#             {"role": "user", "content": context}
#         ],
#         response_format={'type': 'json_object'},
#         temperature=0,
#         top_p=1
#     )

#     return response


### Creating dataset 

In [ ]:
# # Selena

# def make_dataset(file_path: Path):
#         returned_dict = {
#              "context": [],
#              "short_query": [],
#              "medium_query": [],
#              "long_query": [],
#              "keywords": []
#         }
#         # Open and iterate through the .jsonl file
#         with open(file_path, 'r') as file:
#             for line in file:
#                 data = json.loads(line)
#                 context = data[-1]['context']
#                 returned_data = data[1]['choices'][0]['message']['content']
#                 returned_data = json.loads(returned_data)
#                 returned_dict['context'].append(context)
#                 returned_dict['short_query'].append(returned_data['short_query'])
#                 returned_dict['medium_query'].append(returned_data['medium_query'])
#                 returned_dict['long_query'].append(returned_data['long_query'])
#                 returned_dict['keywords'].append(returned_data['keywords'])
#         return returned_dict